In [1]:
import os
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import nltk
from nltk.corpus import stopwords

In [11]:
# 使用pandas读入训练数据
datafile = os.path.join('.','datas','labeledTrainData.tsv')
df = pd.read_csv(datafile,sep="\t",escapechar="\\")
print("Number of reviews: {}".format(len(df)))
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [13]:
df['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [43]:
"""
对影评数做预处理，大概有一下环节：
0,归一化处理，全部转为小写
1，去掉html标签
2，移除标点
3，切分词/token
4,去掉停用词
5，重组为新的句子
"""
def clean_sentence(sentence):
    sentence = sentence.lower() #0,归一化处理，全部转为小写
    sentence = BeautifulSoup(sentence,'html.parser').get_text()#1，去掉html标签
    sentence = re.sub('[\W]',' ',sentence)#2，移除标点
    tokens = nltk.word_tokenize(sentence)#3，切分词/token
    stop_words = set(stopwords.words('english'))
    sentence_words = [i for i in sentence.split() if i not in stop_words]#4,去掉停用词
    return ' '.join(sentence_words)#5，重组为新的句子

example = clean_sentence(df['review'][1])
example

'classic war worlds timothy hines entertaining film obviously goes great effort lengths faithfully recreate h g wells classic book mr hines succeeds watched film appreciated fact standard predictable hollywood fare comes every year e g spielberg version tom cruise slightest resemblance book obviously everyone looks different things movie envision amateur critics look criticize everything others rate movie important bases like entertained people never agree critics enjoyed effort mr hines put faithful h g wells classic novel found entertaining made easy overlook critics perceive shortcomings'

In [44]:
# 清洗数据添加到dataframe里面
df['cleaned_review'] = df.review.apply(clean_sentence)
df.head()

,id,sentiment,review,cleaned_review
0,5814_8,1,With all this stuff going down at the moment w...,stuff going moment mj started listening music ...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin...",classic war worlds timothy hines entertaining ...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film starts manager nicholas bell giving welco...
3,3630_4,0,It must be assumed that those who praised this...,must assumed praised film greatest filmed oper...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80 ex...


In [47]:
"""抽取bag of words 特征（使用sklearn 的CountVectorizer）"""
vectorizer = CountVectorizer(max_features=5000)# 取最大5000个词
train_data_features = vectorizer.fit_transform(df.cleaned_review).toarray()# 抽取特征值
train_data_features.shape

(25000, 5000)

In [53]:
print(len(train_data_features))
print(type(train_data_features))
train_data_features

25000
<class 'numpy.ndarray'>


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [54]:
# 训练分类器
forest = RandomForestClassifier(n_estimators=100) # 构建100棵树
forest = forest.fit(train_data_features,df.sentiment)#训练词向量（词向量的特征值）

In [55]:
# 在训练集上做个predict看看效果
confusion_matrix(df.sentiment,forest.predict(train_data_features))

array([[12500,     0],
       [    0, 12500]], dtype=int64)

In [57]:
# 删除不用的占内存变量
del df
del train_data_features

In [58]:
# 读取测试数据进行预测
# 使用pandas读入训练数据
datafile = os.path.join('.','datas','testData.tsv')
df = pd.read_csv(datafile,sep="\t",escapechar="\\")
print("Number of reviews: {}".format(len(df)))
df['cleaned_review'] = df.review.apply(clean_sentence)
df.head()

Number of reviews: 25000


,id,review,cleaned_review
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main themes mortality nostalgia...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kids saw tonight child loved one point k...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...


In [59]:
test_data_features = vectorizer.fit_transform(df.cleaned_review).toarray()# 提取特征值
test_data_features.shape

(25000, 5000)

In [61]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id,'sentiment':result})
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,1
4,12128_7,0
